# Lithium Capture Single Particle Model Development

I will develop the initial model and governing equations for a single-particle model of an electrochemical lithium capture system. The overarching goal of such technology is to find an efficient method to recover lithium from natural sources such as brine ponds or seawater (difficult, since Li concentration in sea water is very low) for use in battery applications.

The general process for capturing lithium ions (known in the literature as 'electrochemical ion pumping') is as follows [1]:
1. Li cations from brine are intercalated in an $FePO_4$ electrode by applying a current
2. The solution (rid of Li ions) is exchanged for a 'recovery solution'
3. The current is reversed, and intercalated Li ions are released into solution
4. Solution is recovered and replaced with fresh brine; cycle repeats

In this model, I will be considering only step 1, the intercalation of Li cations into an electrode through applying an electric current.

### Note on electrode notation
Lithium recovery electrodes (in this model, $FePO_4$ are typically composed of materials used as cathode electrodes in Li-batteries, since the same properties that make for a suitable cathode make for a suitable lithium recovery electrode (Li selectivity and stability in wet environments [2]).

Therefore, throughout this model I will use the term "cathode" to refer to the lithium recovery electrode and "anode" to refer to the counter electrode.

# #The model domain includes:
- Porous Ag/AgCl anode
- Porous electrolyte separator (? necessary ?)
- Porous FePO4/LiFeO4 cathode

The liquid electrolyte will consist of artifical brine reflecting the typical concentration of Li cations in a Chilean salt pond: $60 10^{-3} M Li^+$

The simulation will assume a constant temperature of 298 K.  We will further assume that the anode, cathode, and electrolyte phases are incompressible (constant molar density).

The state variables are:
- Li intercalation fraction $X_{\rm Li}$ in the $Ag$ anode, $X_{\rm Li,an}$
- Electric potential of the $Ag$ anode.
- Concentration of the Li+ in the electrolyte in the $Ag$ anode.
- Electrolyte electric potential in the $Ag$ anode.
- Concentration of the Li+ in the electrolyte in the separator.
- Electrolyte electric potential in the separator.
- Concentration of the Li+ in the electrolyte in the $FePO_4$ cathode.
- Electrolyte electric potential in the $FePO_4$ cathode.
- Li intercalation fraction $X_{\rm Li}$ in the $FePO_4$, $X_{\rm Li,ca}$
- Electric potential in the $FePO_4$

## Conservation of charge:

### Double layer current:

We begin with conservation of charge at the electrolyte/electrode double layers.  Calling 'el' the electrode phase:
\begin{equation}
    \frac{\partial Q_{\rm el}}{\partial t} = 0 = \pm I_{\rm ext} -i _{\rm Far}A_{\rm surf} - i_{\rm dl}A_{\rm surf}
\end{equation}

which leads eventually to:

\begin{equation}
    i_{\rm dl} = \pm i_{\rm ext}\frac{A_{\rm geo}}{A_{\rm surf}} -i _{\rm Far}
\end{equation}
where the sign on $i_{\rm ext}$ depends on whether we deal with the cathode or the anode.  We consider positive current as the discharge current, which delivers positive charge to the anode ($i_{\rm ext}$), and negative charge to the cathode ($-i_{\rm ext}$). $i_{\rm dl}$ and $i_{\rm Far}$ both move positive charge from the electrode to the electrolyte bulk interior.

where we name the geometric factor $A_{\rm fac}$:
\begin{equation}
    A_{\rm fac} = \frac{A_{\rm geo}}{A_{\rm surf}} = \frac{r_p}{3H_{\rm el}\varepsilon_{\rm AM}}
\end{equation}
where $r_p$ is the active material particle radius, $H_{\rm el}$ is the electrode thickness, and $\varepsilon_{\rm AM}$ is the active material volume fraction in the electrode.


### Double layer potential:

The charge separation at the double layer (C/m$^2$) then evolves as:

\begin{equation}
    \frac{\partial q_{\rm dl}}{\partial t} = i_{\rm dl}
\end{equation}
Defining the charge separation as $q_{\rm dl} = q_{\rm el} - q_{\rm elyte}$ and $\Delta \phi_{\rm dl} = \phi_{\rm el} - \phi_{\rm elyte}$: 

\begin{equation}
    \frac{d\Delta\phi_{\rm dl}}{dt} =\frac{d\left(\phi_{\rm el} - \phi_{\rm elyte}\right)}{dt} = \frac{i_{\rm dl}}{C_{\rm dl}} = \frac{1}{C_{\rm dl}}\left(\pm i_{\rm ext}A_{\rm fac}-i_{\rm Far}\right)
\end{equation}

## Assumptions and boundary conditions.

### In the anode:
We get to pick one electric potential as our reference condition ($\phi = 0$).  Without loss of generality, assume the anode is our reference: 

\begin{equation}
    \phi_{\rm an} = 0 {\rm V.}
\end{equation}
From this, we can calculate the electrolyte electric potential in the anode pores as:

\begin{equation}
    \phi_{\rm elyte,an} = \phi_{\rm an} - \Delta\phi_{\rm dl,an}
\end{equation}
which means that $\Delta\phi_{\rm dl}$ can serve as our state variable.

### In the cathode:
We assume, initially, that the electrolyte across the separator has a fixed resistance:

\begin{equation}
    R_{\rm sep} = \frac{1}{\sigma_{\rm io,elyte}}\left(H_{\rm sep} + 0.5\,H_{\rm an} + 0.5\,H_{\rm ca}\right)
\end{equation}
where $H$ is the component thickness and $\sigma_{\rm io,elyte}$ is the effective ionic conductivity of the electrolyte (incorporating any microstructure effects).

From this, we can calculate the elecric potential of the electrolyte in the cathode, relative to that in the anode:

\begin{equation}
    \phi_{\rm elyte,ca} = \phi_{\rm elyte,an} - i_{\rm ext}R_{\rm sep}
\end{equation}
Finally, we can calculate the cathode active material electric potential, relative to the known electrolyte electric potential and the double layer potential:

\begin{equation}
    \phi_{\rm ca} = \phi_{\rm elyte,ca} + \Delta\phi_{\rm dl,ca}
\end{equation}

Therefore, it is sufficient to store the two double layer potentials $\Delta\phi_{\rm dl,an}$ and $\Delta\phi_{\rm dl,ca}$, which along with $R_{\rm sep}$ and $i_{\rm ext}$ can determine all electric potentials at a given time.

In [35]:
# Inputs:

C_rate = 0.1 # How many charges per hour?

T = 298 #K

#Anode (Ag) properties (much of these need to be updated for my system)
r_p_an = 4e-6 #m
phi_an_0 = 0 #V reference electrode
C_dl_an = 1e4 #F/m2, need to look up
i_o_an = 4.0  #A/m2 need to calculate/look up
n_an = -1
beta_an = 0.5
H_an = 30e-6  #m
density_Ag = 10490 #kg/m3, UPDATED
capacity_Ag = 350 #Ah/kg, need to look up
eps_Ag = .65
dPhi_eq_an = -0.7991 #standard potential for Ag, from Appendix A

#seperator
phi_sep_0 = 1.8  #V, do I actually have a separator?

#Cathode (LFP/FP) properties (much of these need to be updated for my system)
r_p_ca = 0.3e-6 #m
phi_ca_0 = 4.6  #V
C_dl_ca = 1e4 #F/m2
i_o_ca = 100 #A/m2
n_ca = -1
beta_ca = 0.5
H_ca = 50e-6  #m
density_FP = 3056  #kg/m3, UPDATED
capacity_FP = 175  #Ah/kg, ?
eps_FP = 0.65
dPhi_eq_ca = 2.6 #need to look up

# How deep do we want to charge/discharge?
charge_frac = 0.9

### Capacity
We need to calculate the total charge we can store in each electrode (the capacity). This is typically calculated in A-h (1 A-h = 3600 Coulombs). Each active material has a known gravimetric capacity, which can be converted to the electrode capacity (per m$^2$ of battery):

\begin{equation}
    {\rm Cap} = C_{\rm AM}\rho_{\rm AM}\varepsilon_{\rm AM}H_{\rm el}
\end{equation}

The total battery capacity is the minimum of the two electrode capacities.  For a given C-rate, then, the external current density equals the C-rate multiplied by the capacity.

In [36]:
# Initialize:
phi_dl_an_0 = phi_an_0 - phi_sep_0
phi_dl_ca_0 = phi_ca_0 - phi_sep_0


capacity_anode = capacity_Ag*H_an*eps_Ag*density_Ag
capacity_cathode = capacity_FP*H_ca*eps_FP*density_FP
capacity_area = min(capacity_anode,capacity_cathode)


t_final = charge_frac*3600./C_rate
i_ext = C_rate*capacity_area

A_fac_an = r_p_an/3/H_an/eps_Ag
A_fac_ca = r_p_ca/3/H_ca/eps_FP

In [37]:
# Constants
F = 96485
R = 8.3145

In [38]:
import numpy as np
from math import exp

def residual(t,SV):
    dSV_dt = np.zeros_like(SV)
    
    eta_an = SV[0] - dPhi_eq_an
    i_Far_an = i_o_an*(exp(-n_an*F*beta_an*eta_an/R/T)
                      - exp(n_an*F*(1-beta_an)*eta_an/R/T))
    i_dl_an = i_ext*A_fac_an - i_Far_an
    dSV_dt[0] = i_dl_an/C_dl_an
    
    
    eta_ca = SV[1] - dPhi_eq_ca
    i_Far_ca = i_o_ca*(exp(-n_ca*F*beta_ca*eta_ca/R/T)
                      - exp(n_ca*F*(1-beta_ca)*eta_ca/R/T))
    i_dl_ca = -i_ext*A_fac_ca - i_Far_ca
    
    
    dSV_dt[1] = i_dl_ca/C_dl_ca
    
    return dSV_dt

In [39]:
from scipy.integrate import solve_ivp

SV_0 = np.array([phi_dl_an_0, phi_dl_ca_0])

time_span = np.array([0,t_final])

solution = solve_ivp(residual,time_span,SV_0,rtol=1e-6, atol=1e-8)

OverflowError: math range error

In [ ]:
from matplotlib import pyplot as plt
for var in solution.y:
    plt.plot(solution.t,var)
    
plt.legend(['Anode double layer','Cathode double layer'])

For now, the double layer potentials simply go to those values needed to sustain the current density.

## Questions
- Do I have a separator in my system? Or do I just have two electrodes suspended in brine. I left it here for now.
- Does it make sense that I am calling my lithium collection electrode the cathode? Delivering negative current to the cathode should intercalate the Li ions into the cathode, and reversing the current should release them, I think.
- If I don't have a separator, how do I calculate 𝜙elyte,ca? 
- in my python model, if I put dphi_eq_an = -0.7991 (standard potential of Ag, from Appendix A of the textbook), I get a math overflow error in python when I try to run the model.py file. I'm not sure what is going on

## References
1.  A. Battistel, M.S. Palagonia et al., “Electrochemical Methods for Lithium Recovery:A Comprehensive and Critical Review,” Advanced Materials, vol. 32, no. 23, 2020

2. C.P. Lawagon, G.M. Nisola et al., "Li1−xNi0.33Co1/3Mn1/3O2/Ag for electrochemical lithium recovery from brine," Cheimcal Engineering Journal, vol. 348, no. 15, 2018